# Init

In [1]:
# # if used pip install package
# !pip install xgboost
# !pip install lightgbm
# !pip install wget
# !pip install gensim
# !pip install catboost
# !pip install cython

In [2]:
from __future__ import print_function
import os
import sys
import logging
import gc
import wget
import time
import tarfile
import zipfile
import functools
import random
import copy
import pickle
from tqdm import tqdm_notebook, tqdm
from itertools import product, combinations
from scipy.special import comb, perm

import scipy
import numpy as np
import pandas as pd
import catboost as cbt
from glove import *

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import StratifiedKFold, KFold, GroupKFold
from sklearn.linear_model import LogisticRegression,BayesianRidge,SGDClassifier,PassiveAggressiveClassifier,RidgeClassifier
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.ensemble import ExtraTreesClassifier,RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import LinearSVC,NuSVC,SVC
from sklearn.metrics import f1_score, roc_auc_score

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, Embedding, Dropout, Activation, Conv1D, LSTM, GRU  #, CuDNNGRU, CuDNNLSTM
from keras.layers import Bidirectional, GlobalMaxPool1D, GlobalMaxPooling1D, GlobalAveragePooling1D
from keras.layers import Input, Embedding, Dense, Conv2D, MaxPool2D, concatenate
from keras.layers import Reshape, Flatten, Concatenate, Dropout, SpatialDropout1D
from keras.optimizers import Adam
from keras.models import Model, Sequential
from keras import backend as K
from keras.engine.topology import Layer
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.layers import concatenate
from keras.callbacks import *
from keras.optimizers import *
from keras.engine.topology import Layer
from keras.layers import *
from keras.utils import np_utils

import gensim
import xgboost as xgb
import lightgbm as lgb

from joblib import Parallel, delayed

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

Using TensorFlow backend.


In [3]:
# import ray
# ray.init(object_store_memory=int(100e6))
# import modin.pandas as pd
# os.environ["MODIN_ENGINE"] = "ray"  # Modin will use Ray
# os.environ["MODIN_ENGINE"] = "dask"  # Modin will use Dask

In [4]:
class Logger(object):
    def __init__(self, filename='default.log', stream=sys.stdout):
        self.terminal = stream
        self.log = open(filename, 'a')
        
    def write(self, message):
        self.terminal.write(message)
        self.log.write(message)
        
    def flush(self):
        pass

In [5]:
# sys.stdout = Logger("logs/default.log", sys.stdout)
# sys.stderr = Logger("logs/default_err.log", sys.stderr)

In [6]:
DDIR = "data"
UDDIR = "user_data"
UFEDIR = "user_data/feat_data_v05"
UMDIR = "user_data/model_data"
RESDIR = "prediction_result"

In [7]:
UID = "user_id"

# Load data (Only once)

In [8]:
# train_fname = wget.download("https://tesla-ap-shanghai-1256322946.cos.ap-shanghai.myqcloud.com/cephfs/tesla_common/deeplearning/dataset/algo_contest/train_preliminary.zip", out=DDIR)
# test_fname = wget.download("https://tesla-ap-shanghai-1256322946.cos.ap-shanghai.myqcloud.com/cephfs/tesla_common/deeplearning/dataset/algo_contest/test.zip", out=DDIR)

In [9]:
# def myunzip(filename):
#     zFile = zipfile.ZipFile(filename, "r")
#     for fileM in zFile.namelist(): 
#         zFile.extract(fileM, DDIR)
#         print(fileM)
#     zFile.close()

In [10]:
# myunzip(train_fname)
# myunzip(test_fname)

# Utils

In [11]:
def bch_rencol(values, prefix="", suffix=""):
    return list(map(lambda x: f"{prefix}"+"_".join(list(map(lambda y: str(y), x)))+f"{suffix}" 
                    if hasattr(x, "__iter__") and not isinstance(x, str) 
                    else f"{prefix}"+str(x)+f"{suffix}", values))

In [12]:
def mynunique(values):
    return values.nunique(dropna=False)
def getidxmax(x):
    return x.idxmax()[1]
# for time series
def at_len(x):
    return len(x)

def at_sum(x):
    return np.sum(x)

def at_max(x):
    return np.max(x)

def at_min(x):
    return np.min(x)

def at_mean(x):
    return np.mean(x)

def at_range(x):
    return at_max(x) - at_min(x)

def at_nunq(x):
    return len(set(x))

def at_lenDrange(x):
    return at_len(x)/(at_range(x)+1)

def at_lenDnunq(x):
    return at_len(x)/at_nunq(x)

def at_percentile(n):
    def at_percentile_(x):
        return np.percentile(x, n)
    at_percentile_.__name__ = f"at_percentile_{n}"
    return at_percentile_

In [13]:
OP_SET = ["sum", "max", "min", "mean", "std"]
OP_SET1 = ["nunique", "sum", "max", "min", "mean", "std", "median", "skew", at_percentile(0.25), at_percentile(0.75)]
OP_SET2 = ["sum", "max", "min", "mean", "std", "median", "skew", at_percentile(0.25), at_percentile(0.75)]

In [14]:
ID_SET = ["creative_id", "ad_id", "product_id", "product_category", "advertiser_id", "industry", "time", "click_times"]

In [15]:
nesting_level = 0
is_start = None
class Timer:
    def __init__(self):
        self.start = time.time()
        self.history = [self.start]

    def check(self, info):
        current = time.time()
        print(f"[{info}] spend {current - self.history[-1]:0.2f} sec")
        self.history.append(current)

def log(entry):
    global nesting_level
    space = "-" * (4 * nesting_level)
    print(f"{space}{entry}")

def timeit(method, start_log=None):
    @functools.wraps(method)
    def timed(*args, **kw):
        global is_start
        global nesting_level

        if not is_start:
            print()

        is_start = True
        log(f"Start [{method.__name__}]:" + (start_log if start_log else ""))
        log(f'Start time: {time.strftime("%Y-%m-%d %H:%M:%S")}')
        nesting_level += 1

        start_time = time.time()
        result = method(*args, **kw)
        end_time = time.time()

        nesting_level -= 1
        log(f"End   [{method.__name__}]. Time elapsed: {end_time - start_time:0.2f} sec.")
        is_start = False

        return result

    return timed

In [16]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

# Embedding

## Get ID sequence (Only once)

In [17]:
# # read train data
# train_click_log = pd.read_csv(f"{DDIR}/train_preliminary/click_log.csv")
# train_ad = pd.read_csv(f"{DDIR}/train_preliminary/ad.csv")
# # tag
# train_user = pd.read_csv(f"{DDIR}/train_preliminary/user.csv")

In [18]:
# # read test data
# test_click_log = pd.read_csv(f"{DDIR}/test/click_log.csv")
# test_ad = pd.read_csv(f"{DDIR}/test/ad.csv")
# # pd.DataFrame(np.sort(test_click_log[UID].unique()), columns=[UID]).to_csv(f"{DDIR}/test/user.csv", index=False)
# test_user = pd.read_csv(f"{DDIR}/test/user.csv")

In [19]:
# for i in ID_SET:
#     print(i)
#     if i in train_click_log:
#         print(train_click_log[i].min())
#         print(train_click_log[i].max())
        
#         print(test_click_log[i].min())
#         print(test_click_log[i].max())
#     if i in train_ad:
#         print(train_ad[i].min())
#         print(train_ad[i].max())
        
#         print(test_ad[i].min())
#         print(test_ad[i].max())

In [20]:
# train_ad["product_id"] = train_ad["product_id"].replace("\\N", "0").astype(int)
# train_ad["industry"] = train_ad["industry"].replace("\\N", "0").astype(int)

In [21]:
# test_ad["product_id"] = test_ad["product_id"].replace("\\N", "0").astype(int)
# test_ad["industry"] = test_ad["industry"].replace("\\N", "0").astype(int)

In [22]:
# # sort by time, for time series
# train_click_log.sort_values(by="time", inplace=True)
# test_click_log.sort_values(by="time", inplace=True)

In [23]:
# tol_train = pd.merge(train_click_log, train_ad, how="left", on="creative_id")
# tol_test = pd.merge(test_click_log, test_ad, how="left", on="creative_id")

In [24]:
# del train_click_log, test_click_log
# gc.collect()

In [25]:
# @timeit
# def gen_id_series(data, dtyp="train"):
#     for col in tqdm(ID_SET):
#         tmp = data.groupby([UID])[[col]].agg(lambda x: [f"{y}" for y in x])
#         tmp.columns = bch_rencol(tmp.columns)
#         tmp.to_pickle(f"{UDDIR}/imd/{dtyp}_{col}_seq.pkl")
#         tmp = None

In [26]:
# gen_id_series(tol_train, "train")

In [27]:
# gen_id_series(tol_test, "test")

In [28]:
# seqidx_dic = dict()

In [29]:
# for col in ID_SET:
#     train_seq = pd.read_pickle(f"{UDDIR}/imd/train_{col}_seq.pkl")
#     test_seq = pd.read_pickle(f"{UDDIR}/imd/test_{col}_seq.pkl")
#     tol_seq = pd.concat([train_seq, test_seq])
#     seqidx_dic[col] = tol_seq.index

In [30]:
# with open(f"{UDDIR}/imd/seqidx_dic.pkl", "wb") as f:
#     pickle.dump(pickle.dumps(seqidx_dic), f)

## TF-IDF&Count

In [ ]:
@timeit
def gen_tfidf(col, nr_max=1):
    train_seq = pd.read_pickle(f"{UDDIR}/imd/train_{col}_seq.pkl")
    test_seq = pd.read_pickle(f"{UDDIR}/imd/test_{col}_seq.pkl")
    tol_seq = pd.concat([train_seq, test_seq])
    tol_seq[col] = tol_seq[col].apply(lambda x: " ".join(x))
    
    train_seq = None
    test_seq = None
    tfidf_enc = TfidfVectorizer(ngram_range=(1, nr_max), min_df=30)
    tfidf_vec = tfidf_enc.fit_transform(tol_seq[col].values)
    log(f"TF-IDF shape: {tfidf_vec.shape}")

    # save sparse matrix
    scipy.sparse.save_npz(f"{UDDIR}/imd/sparse_tfidf_{col}.npz", tfidf_vec)

In [ ]:
@timeit
def gen_count(col, nr_max=1):
    train_seq = pd.read_pickle(f"{UDDIR}/imd/train_{col}_seq.pkl")
    test_seq = pd.read_pickle(f"{UDDIR}/imd/test_{col}_seq.pkl")
    tol_seq = pd.concat([train_seq, test_seq])
    tol_seq[col] = tol_seq[col].apply(lambda x: " ".join(x))
    
    train_seq = None
    test_seq = None
    count_enc = CountVectorizer(ngram_range=(1, nr_max), min_df=30)
    count_vec = count_enc.fit_transform(tol_seq[col].values)
    log(f"Count shape: {count_vec.shape}")

    # save sparse matrix
    scipy.sparse.save_npz(f"{UDDIR}/imd/sparse_count_{col}.npz", count_vec)

In [ ]:
for col in ID_SET:
    gen_tfidf(col)

In [ ]:
for col in ID_SET:
    gen_count(col)

### SVD

In [ ]:
@timeit
def gen_svd(col, index, prefix="tfidf", n_cpt=64):
    tfidf_vec = scipy.sparse.load_npz(f"{UDDIR}/imd/sparse_{prefix}_{col}.npz")
    if tfidf_vec.shape[1] > n_cpt:
        svd_enc = TruncatedSVD(n_components=n_cpt, n_iter=20, random_state=2020)
        mode_svd = svd_enc.fit_transform(tfidf_vec)
    else:
        n_cpt = tfidf_vec.shape[1]
        mode_svd = tfidf_vec.todense()
    mode_svd = pd.DataFrame(mode_svd)
    mode_svd.columns = [f"{prefix}_svd_{col}_{i}" for i in range(n_cpt)]
    mode_svd.index = index
    # save svd pkl
    mode_svd.to_pickle(f"{UFEDIR}/{prefix}_svd_{col}.pkl")

In [ ]:
with open(f"{UDDIR}/imd/seqidx_dic.pkl", "rb") as f:
    seqidx_dic = pickle.loads(pickle.load(f))

In [ ]:
for col in ID_SET:
    gen_svd(col, seqidx_dic[col])

### Stacking

In [ ]:
train_user = pd.read_csv(f"{DDIR}/train_preliminary/user.csv")
test_user = pd.read_csv(f"{DDIR}/test/user.csv")

In [ ]:
sparse_matrix = None
for col in tqdm(ID_SET):
    print("current filename: ", f"{UDDIR}/imd/sparse_tfidf_{col}.npz", f"{UDDIR}/imd/sparse_count_{col}.npz")
    tfidf_vec = scipy.sparse.load_npz(f"{UDDIR}/imd/sparse_tfidf_{col}.npz")
    count_vec = scipy.sparse.load_npz(f"{UDDIR}/imd/sparse_count_{col}.npz")
    sparse_matrix = scipy.sparse.csr_matrix(scipy.sparse.hstack([sparse_matrix, tfidf_vec]))
    sparse_matrix = scipy.sparse.csr_matrix(scipy.sparse.hstack([sparse_matrix, count_vec]))

In [ ]:
sparse_matrix.shape

In [ ]:
with open(f"{UDDIR}/imd/seqidx_dic.pkl", "rb") as f:
    seqidx_dic = pickle.loads(pickle.load(f))

In [ ]:
comb_ids = list(combinations(ID_SET, 2))

In [ ]:
is_eq = True
for cp in comb_ids:
    if (seqidx_dic[cp[0]] != seqidx_dic[cp[1]]).sum() != 0:
        print(cp)
        is_eq = False
        break
assert is_eq, "Must True"

In [ ]:
X_train = sparse_matrix[seqidx_dic["creative_id"].isin(train_user[UID])]
X_test = sparse_matrix[~seqidx_dic["creative_id"].isin(train_user[UID])]

In [ ]:
# reorder index
re_train_user = pd.DataFrame(index=seqidx_dic["creative_id"][seqidx_dic["creative_id"].isin(train_user[UID])])
re_train_user = pd.merge(re_train_user, train_user, how="left", on=UID)

In [ ]:
# reorder index
re_test_user = pd.DataFrame(index=seqidx_dic["creative_id"][~seqidx_dic["creative_id"].isin(train_user[UID])])
re_test_user = pd.merge(re_test_user, test_user, how="left", on=UID)

In [ ]:
log(X_train.shape)
log(X_test.shape)

In [ ]:
del sparse_matrix

In [ ]:
gc.collect()

#### For age

In [ ]:
tm_clf = Timer()

In [ ]:
random_seed = 2020
num_classes = 10
n_splits = 5
y = re_train_user["age"]
model_zoo = [SGDClassifier(n_jobs=-1,verbose=1), SGDClassifier(loss='log',n_jobs=-1,verbose=1),
             SGDClassifier(loss='modified_huber',n_jobs=-1,verbose=1),
             PassiveAggressiveClassifier(n_jobs=-1,verbose=1),LogisticRegression(C=10),
             RidgeClassifier(solver='lsqr',fit_intercept=False),LinearSVC(verbose=1,max_iter=500),
             BernoulliNB(),MultinomialNB()]
oof = []
count = 0
columns = ['SGD_HINGE', 'SGD_LOG','SGD_HUBER','PAC','LR','RIDGE','LSVC','BNB','MNB']

for model in model_zoo:
    cv_pred_stack = np.zeros((X_train.shape[0],num_classes))
    test_pred_stack = np.zeros((X_test.shape[0],num_classes))
    skf = KFold(n_splits=n_splits,random_state=random_seed)
    
    gc.collect()
    
    if os.path.exists(f"{UDDIR}/imd/tfidf_count_emb_age_{columns[count]}.pkl"):
        tmp = pd.read_pickle(f"{UDDIR}/imd/tfidf_count_emb_age_{columns[count]}.pkl")
    else:
        for index, (train_index, test_index) in enumerate(skf.split(X_train, y)):
            print(index,model)
            
            train_x = test_x = train_y = test_y = None
            gc.collect()
            
            train_x, test_x, train_y, test_y = X_train[train_index], X_train[test_index], y.iloc[train_index], y.iloc[test_index]
            model.fit(train_x,train_y)
            try:
                y_val = model._predict_proba_lr(test_x)
            except:
                y_val = model.predict_proba(test_x)
            cv_pred_stack[test_index] = y_val
            print(y_val.shape)
            try:
                test_pred_stack += model._predict_proba_lr(X_test) / n_splits
            except:
                test_pred_stack += model.predict_proba(X_test) / n_splits
            
                
        print(model,'score:',accuracy_score(y,np.argmax(cv_pred_stack,axis=1)))

        a = pd.DataFrame(cv_pred_stack).add_prefix(columns[count]+"_")
        a[UID] = re_train_user[UID].values
        b = pd.DataFrame(test_pred_stack).add_prefix(columns[count]+"_")
        b[UID] = re_test_user[UID].values
        tmp = a.append(b)
        tmp.to_pickle(f"{UDDIR}/imd/tfidf_count_emb_age_{columns[count]}.pkl")
        
    count += 1
    oof.append(tmp)
    

df_agg = pd.DataFrame()
for i in tqdm(oof):
    df_agg[i.columns] = i
df_agg.to_pickle(f"{UFEDIR}/tfidf_count_emb_age_all.pkl")

In [ ]:
tm_clf.check("clf embedding")

#### For gender

In [ ]:
random_seed = 2020
num_classes = 2
n_splits = 5
y = re_train_user["gender"]
model_zoo = [SGDClassifier(n_jobs=-1,verbose=1), SGDClassifier(loss='log',n_jobs=-1,verbose=1),
             SGDClassifier(loss='modified_huber',n_jobs=-1,verbose=1),
             PassiveAggressiveClassifier(n_jobs=-1,verbose=1),LogisticRegression(C=10),
             RidgeClassifier(solver='lsqr',fit_intercept=False),LinearSVC(verbose=1,max_iter=500),
             BernoulliNB(),MultinomialNB()]
oof = []
count = 0
columns = ['SGD_HINGE', 'SGD_LOG','SGD_HUBER','PAC','LR','RIDGE','LSVC','BNB','MNB']

for model in model_zoo:
    cv_pred_stack = np.zeros((X_train.shape[0],num_classes))
    test_pred_stack = np.zeros((X_test.shape[0],num_classes))
    skf = KFold(n_splits=n_splits,random_state=random_seed)
    
    gc.collect()
    
    if os.path.exists(f"{UDDIR}/imd/tfidf_count_emb_gender_{columns[count]}.pkl"):
        tmp = pd.read_pickle(f"{UDDIR}/imd/tfidf_count_emb_gender_{columns[count]}.pkl")
    else:
        for index, (train_index, test_index) in enumerate(skf.split(X_train, y)):
            print(index,model)

            train_x = test_x = train_y = test_y = None
            gc.collect()

            train_x, test_x, train_y, test_y = X_train[train_index], X_train[test_index], y.iloc[train_index], y.iloc[test_index]
            model.fit(train_x,train_y)
            try:
                y_val = model._predict_proba_lr(test_x)
            except:
                y_val = model.predict_proba(test_x)
            cv_pred_stack[test_index] = y_val
            print(y_val.shape)
            try:
                test_pred_stack += model._predict_proba_lr(X_test) / n_splits
            except:
                test_pred_stack += model.predict_proba(X_test) / n_splits
        print(model,'score:',accuracy_score(y,np.argmax(cv_pred_stack,axis=1)))

        a = pd.DataFrame(cv_pred_stack).add_prefix(columns[count]+"_")
        a[UID] = re_train_user[UID].values
        b = pd.DataFrame(test_pred_stack).add_prefix(columns[count]+"_")
        b[UID] = re_test_user[UID].values
        tmp = a.append(b)
        tmp.to_pickle(f"{UDDIR}/imd/tfidf_count_emb_gender_{columns[count]}.pkl")
        
    count += 1
    oof.append(tmp)
    

df_agg = pd.DataFrame()
for i in tqdm(oof):
    df_agg[i.columns] = i
df_agg.to_pickle(f"{UFEDIR}/tfidf_count_emb_gender_all.pkl")

In [ ]:
tm_clf.check("clf embedding")

## Word2Vec

In [41]:
@timeit
def gen_w2v(col, vesize=128, win=8):
    train_seq = pd.read_pickle(f"{UDDIR}/imd/train_{col}_seq.pkl")
    test_seq = pd.read_pickle(f"{UDDIR}/imd/test_{col}_seq.pkl")
    tol_seq = pd.concat([train_seq, test_seq])
    
    train_seq = None
    test_seq = None
    if os.path.exists(f"{UMDIR}/vectors/w2v{vesize}/w2v_{col}.model"):
        model = gensim.models.Word2Vec.load(f"{UMDIR}/vectors/w2v{vesize}/w2v_{col}.model")
    else:
        model = gensim.models.Word2Vec(sentences=tol_seq[col], size=vesize, window=win, workers=32, sg=1, min_count=1, iter=10)
        model.save(f"{UMDIR}/vectors/w2v{vesize}/w2v_{col}.model")
    
    w2v_list = list()
    
    for it in tqdm(tol_seq[col]):
        tmp = np.zeros(vesize)
        cnt = 0
        for wd in it:
            cnt += 1
            if wd in model:
                tmp += model[wd]
        w2v_list.append(list(tmp/cnt))
    
    w2v_avg = pd.DataFrame(w2v_list)
    w2v_avg.index = tol_seq.index
    w2v_avg.columns = [f"w2v_avg_{col}_{i}" for i in range(vesize)]
    w2v_avg = reduce_mem_usage(w2v_avg)
    w2v_avg.to_pickle(f"{UFEDIR}/w2v_avg_{col}.pkl")

In [ ]:
for col in ID_SET[:-2]:
    gen_w2v(col)

## Doc2Vec

In [ ]:
@timeit
def gen_d2v(col, vesize=300, win=20):
    train_seq = pd.read_pickle(f"{UDDIR}/imd/train_{col}_seq.pkl")
    test_seq = pd.read_pickle(f"{UDDIR}/imd/test_{col}_seq.pkl")
    tol_seq = pd.concat([train_seq, test_seq])
    
    train_seq = None
    test_seq = None
    
    if os.path.exists(f"{UMDIR}/vectors/d2v_{col}.model"):
        model = gensim.models.Doc2Vec.load(f"{UMDIR}/vectors/d2v_{col}.model")
    else:
        docs = [gensim.models.doc2vec.TaggedDocument(words=i[1],tags=[str(i[0])]) for i in tol_seq[col].reset_index().values]
        model = gensim.models.Doc2Vec(documents=docs, size=vesize, window=win, workers=32, iter=10)
        model.save(f"{UMDIR}/vectors/d2v_{col}.model")
    
    d2v_list = list()
    for it, cps in tqdm(tol_seq[col].reset_index().values):
#         if it in model.docvecs:
        d2v_list.append(model.docvecs[str(it)])
#         else:
#             d2v_list.append(model.infer_vector(cps))

    d2v_avg = pd.DataFrame(d2v_list)
    d2v_avg.index = tol_seq.index
    d2v_avg.columns = [f"d2v_avg_{col}_{i}" for i in range(vesize)]
    d2v_avg = reduce_mem_usage(d2v_avg)
    d2v_avg.to_pickle(f"{UFEDIR}/d2v_avg_{col}.pkl")

In [ ]:
for col in ID_SET:
    gen_d2v(col)

## Glove

In [ ]:
@timeit
def gen_glove(col, vesize=300, win=20):
    train_seq = pd.read_pickle(f"{UDDIR}/imd/train_{col}_seq.pkl")
    test_seq = pd.read_pickle(f"{UDDIR}/imd/test_{col}_seq.pkl")
    tol_seq = pd.concat([train_seq, test_seq])
    
    train_seq = None
    test_seq = None
    
    if os.path.exists(f"{UMDIR}/vectors/glove_{col}.model"):
        glove = Glove.load(f"{UMDIR}/vectors/glove_{col}.model")
    else:
        matrix =  Corpus()
        matrix.fit(tol_seq[col].values)
        glove = Glove(no_components=vesize, learning_rate=0.05)
        glove.fit(matrix.matrix,epochs=10,no_threads=30,verbose=1)
        glove.add_dictionary(matrix.dictionary)
        glove.save(f"{UMDIR}/vectors/glove_{col}.model")
        
    ans = []
    for i in tqdm(tol_seq[col].values):
        line = []
        for j in i:
            line.append(glove.word_vectors[glove.dictionary[j]])
        ans.append(np.mean(line,axis=0))
    
    glove_avg = pd.DataFrame(ans)
    glove_avg.index = tol_seq.index
    glove_avg.columns = [f"glove_avg_{col}_{i}" for i in range(vesize)]
    glove_avg = reduce_mem_usage(glove_avg)
    glove_avg.to_pickle(f"{UFEDIR}/glove_avg_{col}.pkl")

In [ ]:
for col in ID_SET:
    gen_glove(col)

## Fasttext (TODO)